# 🎧 YouTube → Whisper-Only Transcription (Colab‑Ready)
This notebook downloads audio from a YouTube URL and transcribes it **using Whisper only** (no YouTube Transcript API).
It uses **[faster-whisper](https://github.com/guillaumekln/faster-whisper)** for speed and lower memory usage.

**What you get:**
- `.txt` plain transcript
- `.srt` subtitles
- `.vtt` subtitles
- `.json` with per-segment timings & text

> Works on Google Colab and local Jupyter. GPU is optional but makes it faster.


## 1) Setup runtime & install dependencies
- Installs `yt-dlp` for downloading the video's audio
- Installs `faster-whisper` (CT2 engine) for transcription
- Ensures `ffmpeg` is available


In [1]:
IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    # Ensure ffmpeg is installed in Colab
    !apt-get -y update > /dev/null
    !apt-get -y install ffmpeg > /dev/null

# Python packages
!pip -q install yt-dlp faster-whisper ffmpeg-python > /dev/null

import os, sys, json, re, shutil, time, math, textwrap, datetime, subprocess
from pathlib import Path

print('Colab:', IN_COLAB)
!ffmpeg -version | head -n 1


Colab: False
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers


## 2) Configuration
Set your YouTube URL and Whisper model size. For many videos `medium` or `large-v3` gives high quality; `small` or `base` is faster.


In [6]:
import os
from pathlib import Path

def get_user_input_simple():
    """Simple interactive input for YouTube URL and parameters"""

    print("🎥 YouTube Video QA Bot - Setup")
    print("=" * 40)

    # Get YouTube URL
    youtube_url = input("📹 Enter YouTube URL: ").strip()
    if not youtube_url:
        print("❌ URL is required!")
        return None

    # Get model size with validation
    model_options = ['tiny', 'base', 'small', 'medium', 'large-v2', 'large-v3']
    print(f"\n🤖 Available Whisper models: {', '.join(model_options)}")
    model_size = input(f"Choose model size [default: small]: ").strip().lower()
    if not model_size or model_size not in model_options:
        model_size = 'small'
        print(f"Using default model: {model_size}")

    # Get language
    language = input("🌍 Enter language code (e.g., 'en', 'es', 'fr') or press Enter for auto-detect: ").strip()
    if not language:
        language = None
        print("Language auto-detection enabled")

    # VAD filter
    vad_input = input("🎤 Enable voice activity detection? [y/N]: ").strip().lower()
    vad_filter = vad_input in ['y', 'yes', 'true', '1']

    # Output directory
    output_dir_input = input(f"📁 Output directory [default: outputs]: ").strip()
    output_dir = Path(output_dir_input) if output_dir_input else Path('outputs')
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        'youtube_url': youtube_url,
        'model_size': model_size,
        'language': language,
        'vad_filter': vad_filter,
        'output_dir': output_dir
    }

# Usage
config = get_user_input_simple()
if config:
    YOUTUBE_URL = config['youtube_url']
    MODEL_SIZE = config['model_size']
    LANGUAGE = config['language']
    VAD_FILTER = config['vad_filter']
    OUTPUT_DIR = config['output_dir']

    print(f"\n✅ Configuration saved:")
    print(f"   URL: {YOUTUBE_URL}")
    print(f"   Model: {MODEL_SIZE}")
    print(f"   Language: {LANGUAGE or 'Auto-detect'}")
    print(f"   VAD Filter: {VAD_FILTER}")
    print(f"   Output: {OUTPUT_DIR.resolve()}")

🎥 YouTube Video QA Bot - Setup

🤖 Available Whisper models: tiny, base, small, medium, large-v2, large-v3

✅ Configuration saved:
   URL: https://www.youtube.com/watch?v=unMXvnY2FNM
   Model: medium
   Language: en
   VAD Filter: False
   Output: /Volumes/Untitled/Lessons_By_Week/Youtube_Project/Working_Pipelines/outputs


In [8]:
# === Required input ===
YOUTUBE_URL = 'https://www.youtube.com/watch?v=-sxUQk5pxX0'  # paste your URL here

# Whisper model options commonly used with faster-whisper:
# tiny | base | small | medium | large-v2 | large-v3 | distil-small.en | distil-large-v3
MODEL_SIZE = 'small'              # change to 'medium' or 'large-v3' for better quality
LANGUAGE = None                   # e.g., 'en' to force English; None = autodetect
DEVICE = 'auto'                   # 'auto', 'cpu', or 'cuda'
COMPUTE_TYPE = 'auto'             # 'auto', 'int8', 'int8_float16', 'float16', 'float32'
VAD_FILTER = True                 # optional voice activity detection for cleaner segments

# Output directory (created if missing)
OUTPUT_DIR = Path('outputs')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print('OUTPUT_DIR:', OUTPUT_DIR.resolve())


OUTPUT_DIR: /content/outputs


## 3) Helper functions
- Sanitize titles
- Download audio with `yt-dlp` (M4A)
- Format SRT/VTT


In [3]:
import re
from pathlib import Path
import subprocess
from typing import List, Tuple

def slugify(value: str, max_len: int = 80) -> str:
    value = re.sub(r'[\s/\\:]+', '-', value.strip())
    value = re.sub(r'[^a-zA-Z0-9\-_.]+', '', value)
    return value[:max_len] or 'audio'

def run(cmd: List[str]):
    print('$', ' '.join(cmd))
    return subprocess.run(cmd, check=True)

def download_youtube_audio(url: str, out_dir: Path) -> Tuple[Path, str]:
    '''Downloads best M4A audio with yt-dlp and returns (audio_path, video_title).'''
    out_dir.mkdir(parents=True, exist_ok=True)
    template = str(out_dir / '%(title)s [%(id)s].%(ext)s')
    cmd = [
        'yt-dlp',
        '-f', 'bestaudio[ext=m4a]/bestaudio/best',
        '--extract-audio',
        '--audio-format', 'm4a',
        '-o', template,
        url,
    ]
    print('Downloading audio...')
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(result.stdout)
        print(result.stderr)
        raise RuntimeError('yt-dlp failed. See logs above.')

    m4as = sorted(out_dir.glob('*.m4a'), key=lambda p: p.stat().st_mtime, reverse=True)
    if not m4as:
        raise FileNotFoundError('No .m4a file found after yt-dlp run.')
    audio_path = m4as[0]
    title = audio_path.stem
    title = re.sub(r'\s*\[[A-Za-z0-9_-]{6,}\]$', '', title).strip()
    return audio_path, title

def format_timestamp_srt(seconds: float) -> str:
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    ms = int((seconds - int(seconds)) * 1000)
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

def format_timestamp_vtt(seconds: float) -> str:
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    ms = int((seconds - int(seconds)) * 1000)
    return f"{h:02d}:{m:02d}:{s:02d}.{ms:03d}"

def write_srt(segments, path: Path):
    lines = []
    for i, seg in enumerate(segments, start=1):
        start = format_timestamp_srt(seg['start'])
        end = format_timestamp_srt(seg['end'])
        text = seg['text'].strip()
        lines.append(str(i))
        lines.append(f"{start} --> {end}")
        lines.append(text)
        lines.append('')
    path.write_text('\n'.join(lines), encoding='utf-8')
    print('Wrote SRT:', path)

def write_vtt(segments, path: Path):
    lines = ['WEBVTT', '']
    for seg in segments:
        start = format_timestamp_vtt(seg['start'])
        end = format_timestamp_vtt(seg['end'])
        text = seg['text'].strip()
        lines.append(f"{start} --> {end}")
        lines.append(text)
        lines.append('')
    path.write_text('\n'.join(lines), encoding='utf-8')
    print('Wrote VTT:', path)


## 4) Transcribe with **Whisper (faster‑whisper)**
This step loads the chosen model, transcribes the downloaded audio, and saves outputs.


In [4]:
# --- Minimal config (safe defaults) ---

# Model choice: tiny | base | small | medium | large-v3
MODEL_SIZE = globals().get("MODEL_SIZE", "small")

# Language: set to e.g. "en" to force English, or None to auto-detect
LANGUAGE = globals().get("LANGUAGE", None)

# Voice Activity Detection (helps skip non-speech)
VAD_FILTER = globals().get("VAD_FILTER", True)

# Output folder
from pathlib import Path
OUTPUT_DIR = Path(globals().get("OUTPUT_DIR", "outputs"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ---- Device & compute type auto-detect ----
def _has_cuda():
    try:
        import torch
        return torch.cuda.is_available()
    except Exception:
        return False

if _has_cuda():
    DEVICE = "cuda"
    # float16 is usually fastest on GPU; faster-whisper supports "float16"
    COMPUTE_TYPE = "float16"
else:
    DEVICE = "cpu"
    # On CPU, "int8" or "int8_float16" saves RAM; "float32" is the most compatible
    COMPUTE_TYPE = "int8"

print(f"MODEL_SIZE={MODEL_SIZE}, LANGUAGE={LANGUAGE}, VAD_FILTER={VAD_FILTER}")
print(f"DEVICE={DEVICE}, COMPUTE_TYPE={COMPUTE_TYPE}, OUTPUT_DIR={OUTPUT_DIR.resolve()}")


MODEL_SIZE=medium, LANGUAGE=en, VAD_FILTER=False
DEVICE=cpu, COMPUTE_TYPE=int8, OUTPUT_DIR=/Volumes/Untitled/Lessons_By_Week/Youtube_Project/Working_Pipelines/outputs


In [7]:
from faster_whisper import WhisperModel
import json

def transcribe_whisper(
    audio_path: Path,
    model_size: str = 'small',
    language: str | None = None,
    device: str = 'auto',
    compute_type: str = 'auto',
    vad_filter: bool = True,
):
    print(f'Loading faster-whisper model: {model_size} (device={device}, compute_type={compute_type})')
    model = WhisperModel(model_size, device=device, compute_type=compute_type)
    segments, info = model.transcribe(
        str(audio_path),
        language=language,
        vad_filter=vad_filter,
        beam_size=5,
        best_of=5,
        temperature=0.0,
    )
    collected = []
    for seg in segments:
        collected.append({
            'id': seg.id,
            'start': seg.start,
            'end': seg.end,
            'text': seg.text.strip(),
        })
    print('Detected language:', info.language, ' (prob:', round(info.language_probability, 3), ')')
    return collected, info

# 1) Download audio
audio_path, title = download_youtube_audio(YOUTUBE_URL, OUTPUT_DIR)
base = slugify(title)
print('Title:', title)
print('Audio:', audio_path)

# 2) Transcribe
segments, info = transcribe_whisper(
    audio_path,
    model_size=MODEL_SIZE,
    language=LANGUAGE,
    device=DEVICE,
    compute_type=COMPUTE_TYPE,
    vad_filter=VAD_FILTER,
)

# 3) Save outputs
txt_path = OUTPUT_DIR / f'{base}.txt'
srt_path = OUTPUT_DIR / f'{base}.srt'
vtt_path = OUTPUT_DIR / f'{base}.vtt'
json_path = OUTPUT_DIR / f'{base}.segments.json'

full_text = '\n'.join(seg['text'] for seg in segments)
txt_path.write_text(full_text, encoding='utf-8')
print('Wrote TXT:', txt_path)

write_srt(segments, srt_path)
write_vtt(segments, vtt_path)

with open(json_path, 'w', encoding='utf-8') as f:
    json.dump({
        'title': title,
        'source': str(audio_path),
        'model': MODEL_SIZE,
        'language_detected': info.language,
        'language_probability': info.language_probability,
        'segments': segments,
    }, f, ensure_ascii=False, indent=2)
print('Wrote JSON:', json_path)

print('\nAll done! You can download the files in the \'outputs/\' folder.')


Title: We Can’t Afford Groceries, Yet Billionare Wealth is Exploding
Audio: outputs/We Can’t Afford Groceries, Yet Billionare Wealth is Exploding [unMXvnY2FNM].m4a
Loading faster-whisper model: medium (device=cpu, compute_type=int8)
Detected language: en  (prob: 1 )
Wrote TXT: outputs/We-Cant-Afford-Groceries-Yet-Billionare-Wealth-is-Exploding.txt
Wrote SRT: outputs/We-Cant-Afford-Groceries-Yet-Billionare-Wealth-is-Exploding.srt
Wrote VTT: outputs/We-Cant-Afford-Groceries-Yet-Billionare-Wealth-is-Exploding.vtt
Wrote JSON: outputs/We-Cant-Afford-Groceries-Yet-Billionare-Wealth-is-Exploding.segments.json

All done! You can download the files in the 'outputs/' folder.


## 5) (Optional) Transcribe a local audio/video file
If you already have an audio or video file, you can upload it here and transcribe without YouTube.


In [ ]:
if 'IN_COLAB' in globals() and IN_COLAB:
    from google.colab import files  # type: ignore
    print('Upload a file to transcribe (audio/video):')
    up = files.upload()
    if up:
        local_name = list(up.keys())[0]
        local_path = Path(local_name)
        segments2, info2 = transcribe_whisper(
            local_path,
            model_size=MODEL_SIZE,
            language=LANGUAGE,
            device=DEVICE,
            compute_type=COMPUTE_TYPE,
            vad_filter=VAD_FILTER,
        )
        base2 = slugify(local_path.stem)
        (OUTPUT_DIR / f'{base2}.txt').write_text('\n'.join(s['text'] for s in segments2), encoding='utf-8')
        write_srt(segments2, OUTPUT_DIR / f'{base2}.srt')
        write_vtt(segments2, OUTPUT_DIR / f'{base2}.vtt')
        with open(OUTPUT_DIR / f'{base2}.segments.json', 'w', encoding='utf-8') as f:
            json.dump({
                'title': local_path.stem,
                'source': str(local_path),
                'model': MODEL_SIZE,
                'language_detected': info2.language,
                'language_probability': info2.language_probability,
                'segments': segments2,
            }, f, ensure_ascii=False, indent=2)
        print("Saved outputs for uploaded file in 'outputs/'")


## 6) Download your files
Run this cell to list generated files and show quick download links (works in Colab).


In [17]:
from pathlib import Path

print('Generated files in outputs/:')
for p in sorted(Path('outputs').glob('*')):
    print('-', p)

if 'IN_COLAB' in globals() and IN_COLAB:
    from google.colab import files  # type: ignore
    print('\nClick to download from the file browser (left sidebar) or use:')
    # Example: trigger a download of the first .txt file (uncomment next line)
    # files.download(str(next(Path('outputs').glob('*.txt'), None)))


Generated files in outputs/:
- outputs/How Reno & FFK Changed Their Tune on Nigeria’s Christian Genocide [VM-iPVE8qsA].m4a
- outputs/How-Reno--FFK-Changed-Their-Tune-on-Nigerias-Christian-Genocide.segments.json
- outputs/How-Reno--FFK-Changed-Their-Tune-on-Nigerias-Christian-Genocide.srt
- outputs/How-Reno--FFK-Changed-Their-Tune-on-Nigerias-Christian-Genocide.txt
- outputs/How-Reno--FFK-Changed-Their-Tune-on-Nigerias-Christian-Genocide.vtt
- outputs/T-A-A-Foundation-son.segments.json
- outputs/T-A-A-Foundation-son.srt
- outputs/T-A-A-Foundation-son.txt
- outputs/T-A-A-Foundation-son.vtt
- outputs/Trump Threatens to Invade Nigeria ｜ Dele Farotimi Responds POWERFULLY [-sxUQk5pxX0].m4a
- outputs/Trump-Threatens-to-Invade-Nigeria--Dele-Farotimi-Responds-POWERFULLY.segments.json
- outputs/Trump-Threatens-to-Invade-Nigeria--Dele-Farotimi-Responds-POWERFULLY.srt
- outputs/Trump-Threatens-to-Invade-Nigeria--Dele-Farotimi-Responds-POWERFULLY.txt
- outputs/Trump-Threatens-to-Invade-Nigeria--Del